In [1]:
import os
import sys
src_path = sys.path[0].replace("scripts", "src")
# data_path = sys.path[0].replace("notebooks", "data")
if src_path not in sys.path:
    sys.path.append(src_path)

out_path = sys.path[0].replace("scripts", "output")

import warnings
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
import time

from profile_extraction import ProfileExtraction
from data_loader import UCSD_dataloader
from data_pool import DataPool

from battery_model import *
from optimizer import *
from predictor import Predictor_tou_SDGE_DA, Predictor_tou_CAISO
from exp_manager import ExperimentManager
import copy

g:\Anaconda\Anaconda3\envs\base38\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
exp_suffix = "Mar2023"
exp_folder = os.path.join(out_path, "experiments", exp_suffix)
assert os.path.exists(exp_folder)

In [3]:
tou_pred = Predictor_tou_SDGE_DA(None)
tou = tou_pred.get_prediction(datetime(2019,1,1,0,0), 96, 0.25)

opt_params_sample = {
    "K": 96,
    "delta": 0.25,
    "S": 1, # number of scenarios (energy profiles). can omit, default 1
    "S_prob": None, 
    # if S != 1, params in "loads", "ev" should be list of length S
    ### loads
    "load_bld": 5 * np.sin(np.linspace(0, 2*np.pi, 96)) + 10,
    "load_pv": np.maximum(0, -np.linspace(-6,6,96)**2+20),
    "energy_price_buy": tou, # $/kWh
    "energy_price_sell": 0.6, # if None, sell is not allowed. can omit
    ### demand charge
    "dc_price": 0.6, # $/day. ref: 18 $/mon
    "dc_prev_max": None, # track p_grid_max in the same billing cycle. can omit
    "p_grid_max": "1.5",    # "1.5" means 1.5 x max(load_bld), a float, e.g., 1000, means 1000 kW, default: None
    ### battery (here I only consider one battery)
    """ Battery params will be provided by battery object """
    # "bat_capacity": None, # if none, capacity is optimized
    # "bat_p_max": 3, # i.e., capacity (kWh) / p_bat_max (kW) = 3 (h)
    # "bat_p_min": 3, # can omit, then p_bat_min = p_bat_max
    # "bat_price": 100, # $/kWh, ref: Tesla Powerwall
    # "bat_efficacy": 0.98, 
    # "bat_life_0": 3650, # days.
    # "bat_cycle_0": 5000, # cycles in lifetime
    "deg_model_opt": "throughput",  # valid values: "throughput", "Crate", "rainflow", "DOD"
    "deg_thres_opt": None,
    "deg_lambda_opt": None,
    "bat_soc_0": 0, # B0, BT have to be fractions (SoC indeed). 0.5 if omit FIXME: if 0.5, MSC can take advantage of this
    "bat_soc_K": None, # if None, default is the same as bat_soc_0
    ### EVs
    "ev_I": 20,
    "ev_ta": np.linspace(0, 48, 20), # ta, td can be floats
    "ev_td": np.linspace(36, 96, 20),
    "ev_e_init": np.array([0]*20),
    "ev_e_targ": np.array([10]*20),
    "ev_capacity": None, # can omit, default as e_targ (useful only when aloow discharge)
    "ev_p_max": 6.6,
    "ev_p_min": 0, # can omit, default as 0
    "ev_efficacy": 0.98,
    "ev_charge_rule": "optimal",
    "ev_charge_rule_default": "unif"
}


bat_params_sample ={
    "bat_capacity": None,
    "bat_p_max": 3, # i.e., capacity (kWh) / p_bat_max (kW) = 3 (h)
    "bat_p_min": 3, # can omit, then p_bat_min = p_bat_max
    "bat_price": 150, # $/kWh (old: 1000, ref: Tesla Powerwall)
    "bat_efficacy": 0.98, 
    "bat_life_0": 3650, # days.
    "bat_cycle_0": 3000, # cycles in lifetime
    # battery degradation params
    "deg_model": "throughput",  
        # valid values: "throughput", "Crate", "rainflow", "DOD"
    #   [1. degradation ~ high C-rate]
    "deg_Crate_thres": (0.25, 0.25, 0.25, 0.25),
    "deg_Crate_lambda": (0.8, 1, 1.5, 2),
    #   [2. degradation ~ large cycle depth]
    "deg_rainflow_thres": (0.2, 0.2, 0.2, 0.4),
    "deg_rainflow_lambda": (0.6, 1, 1.5, 1.8),
    #   [3. degradation ~ low SoE range]
    "deg_DOD_thres": (0.5, 0.2, 0.2, 0.1),
    "deg_DOD_lambda": (1.3, 1.15, 0.85, 0.6),
}

In [4]:
class EnS_ExperimentManager(ExperimentManager):
    def run_one_trial(self, params, save_fn):
        self.cache = {}
        # params: keys: "pv_to_bld", "ev_to_bld", "ev_charge_rule", "ev_p_max"

        pv_to_bld = params["pv_to_bld"]
        ev_to_bld = params["ev_to_bld"]
        ev_charge_rule = params["ev_charge_rule"]
        ev_p_max = params["ev_p_max"]

        ev_source = "OSLER"
        incl_ev, ev_how_to = True, "unif"
        # if ev_to_bld == 0 or ev_to_bld is None:
        #     ev_source, ev_to_bld = None, None
        #     incl_ev, ev_how_to = False, None

        # load data set
        loader = UCSD_dataloader
        loaded = loader(tstart=datetime(2019,1,1,0,0), tend=datetime(2020,1,1,0,0), delta=0.25, 
                        bld="Hopkins", pv="Hopkins", ev=ev_source, pv_to_bld=pv_to_bld, ev_to_bld=ev_to_bld, Pmax=ev_p_max)
        data = loaded.get_data()

        # extract representative profiles
        tstart, tend = datetime(2019, 1, 1, 0, 0), datetime(2020, 1, 1, 0, 0)
        pe = ProfileExtraction(data=data, tstart=tstart, tend=tend)


        """ PART 1: EXTRACT REPRESENTATIVE PROFILE """

        # rp is a [RepProfiles] object, see src/profile/extraction.py
        rp = pe.rep_profile_extraction(
            alg = "sample", K = 20, 
            incl_ev = incl_ev, ev_how_to = ev_how_to,
            dist_metric = "l2_center", importance_func = None, 
            rand_seed=2023)
        
        # stats = rp.stats
        
        # convert profiles to params -> for battery optimizer
        rep_profiles_params = rp.profiles_to_params()
        
        # assume the ev battery capacity is 50 kWh
        ev_e_targ = rep_profiles_params["ev_e_targ"]
        cap = 50    
        ev_capacity = [np.maximum(cap, ev_e_targ[s]) for s in range(len(ev_e_targ))]
        rep_profiles_params["ev_capacity"] = ev_capacity

        
        """ PART 2: USE REP PROFILES FOR SIZING """
        
        opt_params = copy.deepcopy(opt_params_sample)
        opt_params.update(rep_profiles_params)
        bat_params = copy.deepcopy(bat_params_sample)

        # keep unchanged in this experiment
        strategy = "optimal"
        bat_capacity = None
        deg_model_opt = "rainflow"
        deg_model = "DOD"
        ev_charge_rule = ev_charge_rule
        p_grid_max = None

        bat_params["bat_capacity"] = bat_capacity
        bat_params["deg_model"] = deg_model
        
        opt_params["bat_capacity"] = bat_capacity
        opt_params["deg_model_opt"] = deg_model_opt
        opt_params["ev_charge_rule"] = ev_charge_rule
        opt_params["p_grid_max"] = p_grid_max

        opt_params["ev_p_min"] = copy.deepcopy(opt_params["ev_p_max"])


        b_sample = Battery_base(bat_params)
        opt = Battery_optimizer(battery=b_sample)
        
        self.cache["opt"] = opt
        
        sol = opt.optimize_battery_size(opt_params, strategy=strategy, mute=True)
        df = sol.sol_summary()
        

        if self.save:
            # Attention: we only save the sizing solution, which includes rep prefiles info
            sol.save(save_fn=save_fn, save_path=self.save_path)
        
        
        stats = dict(df.T["All"])

        

        return stats

In [6]:
log_fn = os.path.join(exp_folder, "LOG-EV_battery_rep.xlsx")
save_path = os.path.join(exp_folder, "EXP-extract-size")
em = EnS_ExperimentManager(log_fn=log_fn, save_path=save_path, save=True, exp_prefix="NEW-V2B")
var_keys = ["pv_to_bld", "ev_to_bld", "ev_charge_rule", "ev_p_max"]
em.run(keys=var_keys, num_trials=10)

!!!!!!!!!! MISSING VALUES !!!!!!!!!! || [bld] has [4] missing values
!!!!!!!!!! MISSING VALUES !!!!!!!!!! || [pv] has [4] missing values
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 0 sessions
Done, trial 1
DONE
